### Importing necessary Libraries for handling data

In [0]:
import numpy as np
import pandas as pd
import math as mt

### Converting Training Sequences to its corresponding PWMs (Position Weight Matrix)

In [0]:
trainSq = pd.read_csv('/content/drive/My Drive/Colab Notebooks/trainSeq.csv')

In [0]:
train_r = list(trainSq['Seq'])

In [0]:
pos = []
neg = []
indx = []
for i in range(len(trainSq['ATP'])):
    if trainSq['ATP'][i] == 1:
        pos.append(trainSq['Seq'][i])
        indx.append(i)
    else:
        neg.append(trainSq['Seq'][i])

print(len(pos),len(neg))

In [0]:
scr = {'A':1,'G':2,'I':3,'L':4,'V':5,'M':6,'F':7,'W':8,'P':9,'C':10,'S':11,'T':12,'Y':13,'N':14,'Q':15,'H':16,'K':17,'R':18,'D':19,'E':20,'X':21}

In [0]:
scr_raw_p = np.full((21,17),0,dtype=float)
scr_raw_n = np.full((21,17),0,dtype=float)

In [0]:
for i in range(len(train_r)):
    if(i in indx):
        for j in range(17):
            scr_raw_p[scr[train_r[i][j].upper()]-1][j] += 1
    else:
        for j in range(17):
            scr_raw_n[scr[train_r[i][j].upper()]-1][j] += 1

In [0]:
scr_raw_p = (scr_raw_p+(1/21))/(len(pos)+1)
scr_raw_p = (np.log2(scr_raw_p/0.05))/mt.log2(10)
scr_raw_p = 1/(1+2**-scr_raw_p)

In [0]:
scr_raw_n = (scr_raw_n+(1/21))/(len(neg)+1)
scr_raw_n = (np.log2(scr_raw_n/0.05))/mt.log2(10)
scr_raw_n = 1/(1+2**-scr_raw_n)

In [0]:
train = list()
for i in range(len(train_r)):
    l = list()
    if(i in indx):
        for j in range(len(train_r[i])):
            l.append(scr_raw_p[scr[train_r[i][j].upper()]-1][j])
    else:
        for j in range(len(train_r[i])):
            l.append(scr_raw_n[scr[train_r[i][j].upper()]-1][j])
        
    train.append(l)

In [0]:
# lfm = []
# for i in train_r:
#     tfm = []
#     for j in i:
#         kfm = [0]*21
#         kfm[scr[j.upper()]-1] = 1
#         tfm = tfm + kfm
#     lfm += [tfm]
# len(lfm[0])

### Training classifier on training PWMs

In [0]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix, matthews_corrcoef

In [0]:
kf = KFold()

In [0]:
Xa = pd.DataFrame(train)
# Xb = pd.DataFrame(lfm)
# frames = [Xa,Xb]
# X = pd.concat(frames,axis=1,ignore_index=True)
X = Xa
y = pd.DataFrame(list(trainSq['ATP']))

In [0]:
X1 = X.to_numpy()
y1 = y.to_numpy()

In [0]:
y1 = y1.reshape(1,len(trainSq))

In [0]:
clf = SVC(class_weight='balanced')

In [0]:
y1 = y1[0]

### Five Fold Cross Validation

In [0]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y1[train_index], y1[test_index]

    clf.fit(X_train, y_train)
    print(clf.score(X_test,y_test))

    y_pred = clf.predict(X_test)
    print(matthews_corrcoef(y_test, y_pred))
    disp = plot_confusion_matrix(clf, X_test, y_test)
    print(disp.confusion_matrix)

### Importing Testing Sequences and Converting them to PWMs

In [0]:
testSq = pd.read_csv('/content/drive/My Drive/Colab Notebooks/testSeq.csv')

In [0]:
test_r = list(testSq['0'])

In [0]:
scr_raw_test = np.full((21,17),0,dtype=float)

In [0]:
for i in range(len(test_r)):
    for j in range(17):
        scr_raw_test[scr[test_r[i][j].upper()]-1][j] += 1

In [0]:
scr_raw_test = (scr_raw_test+(1/21))/(13024)
scr_raw_test = (np.log2(scr_raw_test/0.05))/mt.log2(10)
scr_raw_test = 1/(1+2**-scr_raw_test)

In [0]:
test = list()
for i in range(len(test_r)):
    l = list()
    for j in range(len(test_r[i])):
        l.append(scr_raw_test[scr[test_r[i][j].upper()]-1][j])
        
    test.append(l)

In [0]:
# lfm = []
# for i in test_r:
#     tfm = []
#     for j in i:
#         kfm = [0]*21
#         kfm[scr[j.upper()]-1] = 1
#         tfm = tfm + kfm
#     lfm += [tfm]
# len(lfm)

In [0]:
Xa = pd.DataFrame(test)
# Xb = pd.DataFrame(lfm)
# frames = [Xa,Xb]
# X_test = pd.concat(frames,axis=1,ignore_index=True)
X_test = Xa

In [0]:
X_test = X_test.to_numpy()

In [0]:
res = clf.predict(X_test)

In [0]:
k = 0
for i in res:
    if i == -1:
        k += 1
print(k,13023-k)

### Writing the final predicted results in ans.csv

In [0]:
import csv

In [0]:
f = open("/content/drive/My Drive/Colab Notebooks/ans.csv", "w")
writer = csv.DictWriter(f, fieldnames=["ID", "Lable"])
writer.writeheader()
f.close()

In [0]:
id = 10001
with open('/content/drive/My Drive/Colab Notebooks/ans.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    for row in res:
        writer.writerow([id, row])
        id = id + 1
f.close()